In [ ]:
import pandas as pd
import tushare as ts
ts.set_token('')
pro = ts.pro_api()
%pylab inline

In [ ]:
widebased_dict = {
    '中证100' : '512910.SH',
    '上证50'  : '510050.SH',
    '深证F120': '159910.SZ',
    '沪深300' : '510300.SH',
    '中证500' : '510500.SH',
    '上证红利' : '512880.SH',
    '中小300' : '159907.SZ',
    '创业ETF' : '159952.SZ'
}

overseas_dict = {
    '标普500' : '513500.SH',
    '纳指ETF' : '159941.SZ',
    '日经225' : '513000.SH',
    '德国30' : '513030.SH',
    '恒生ETF' : '159920.SZ',
}

industry_dict = {
    '消费' : '159928.SZ',
    '工业' : '159953.SZ',
    '医药' : '159938.SZ',
    '传媒' : '512980.SH',
    '金融' : '159940.SZ',
    '可选' : '159936.SZ',
    '环保' : '512580.SH',
    '信息技术' : '159939.SZ',
    '证券' : '159952.SZ',
    '军工' : '512680.SH',
    '房地产' : '512200.SH',
    '人工智能' : '161631.SZ',
}

alldict = dict(widebased_dict, **overseas_dict)
alldict = dict(alldict, **industry_dict)
# dict(widebased_dict.items()+overseas_dict.items()+industry_dict.items())

print(alldict)

ts_code = alldict['工业']
start='20170101'
end='20190820'
holidays = [datetime.datetime(2015, 2, 19), 
            datetime.datetime(2015, 4, 5), 
            datetime.datetime(2015, 5, 2), 
            datetime.datetime(2015, 10, 5), 
            datetime.datetime(2016, 2, 8), 
            datetime.datetime(2016, 4, 4), 
            datetime.datetime(2016, 5, 2), 
            datetime.datetime(2016, 10, 3), 
            datetime.datetime(2017, 1, 2), 
            datetime.datetime(2017, 1, 30), 
            datetime.datetime(2017, 4, 3), 
            datetime.datetime(2017, 5, 1), 
            datetime.datetime(2017, 5, 29), 
            datetime.datetime(2017, 10, 2), 
            datetime.datetime(2018, 1, 1), 
            datetime.datetime(2018, 2, 19), 
            datetime.datetime(2018, 4, 30), 
            datetime.datetime(2018, 6, 18), 
            datetime.datetime(2018, 9, 24), 
            datetime.datetime(2018, 10, 1), 
            datetime.datetime(2018, 12, 31), 
            datetime.datetime(2019, 2, 4), 
            datetime.datetime(2019, 2, 4), 
            datetime.datetime(2019, 2, 5), 
            datetime.datetime(2019, 2, 6), 
            datetime.datetime(2019, 5, 1), 
            datetime.datetime(2019, 10, 2)]


In [ ]:
def plotfund(ts_code, start, end):
    df = pro.fund_daily(ts_code=ts_code, start_date=start, end_date=end)
    df['ma5']=df['close'].rolling(window=5).mean()
    df['ma21']=df['close'].rolling(window=21).mean()
    df1 = df.iloc[::-1]
    # df1[['close','ma5','ma21']].plot(kind='line',use_index=False, figsize=(10,5))
    df1[['ma21']].plot(kind='line',use_index=False, figsize=(10,5))
    
plotfund(ts_code, start, end)

In [ ]:
def ValidStartdate(start, df):
    le = len(df['trade_date'])
    start = df['trade_date'][le-1]
    return start

def ValidDate(date, df):
    ifin = df['trade_date'].isin([date])
    hasone = False
    for flag in ifin:
        if flag is True:
            hasone = True
            break
    return hasone
# fixed_investment 每周定投数额
# feesrate 交易费率
def WeeklyInvestments(ts_code, start, end, weekmask = 'Mon', fixed_investment = 1000, feesrate = 0.00025, printable = False):
    df = pro.fund_daily(ts_code=ts_code, start_date=start, end_date=end)
    start = ValidStartdate(start, df)
    # print(df)
    dates = pd.bdate_range(start=start, end=end, weekmask = weekmask, holidays = holidays, freq='C').strftime("%Y%m%d").tolist()
    num = len(dates)
    totalcount = 0
    if printable:
        print(num)
    initvalue = float(df.loc[df['trade_date'].isin([dates[0]]), 'close'])
    for i in range(num):
        target = week_investment*(i+1) # 每周定投固定数额累计
        if ValidDate(dates[i], df):
            value = float(df.loc[df['trade_date'].isin([dates[i]]), 'close']) # 当前收盘值
        else:
            if printable:
                print(dates[i])
            continue
        count = int(week_investment/value+0.5) # 当期应购数量
        totalcount = totalcount + count # 当前已拥有数量
        equity = totalcount*value # 当前净值
        income = equity - target - target*feesrate # 收益绝对值
        incomepercent = income / target * 100 # 定投收益百分比
        initpercent = (value-initvalue)/initvalue *100 # 相对初始收益百分比
        if printable:
            print("%2d %s: %5d\t%.3f\t%d\t%.2f  \t%.2f    \t%.2f%%\t\t%.2f%%"%(i, dates[i], target, value, count, equity, income, incomepercent, initpercent))
    return incomepercent, start
        
WeeklyInvestments(ts_code, start, end, printable = True)

In [ ]:
def ValidMonthDate(date, df):
    while True:
        ifin = df['trade_date'].isin([date])
        hasone = False
        for flag in ifin:
            if flag is True:
                hasone = True
                break
        if hasone is True:
            break
        else:
            date = str(int(date) - 1)
            # print("new date: ", date)   
    return date
def MonthlyInvestments(ts_code, start, end, fixed_investment = 1000, feesrate = 0.00025, printable = False):
    df = pro.fund_daily(ts_code=ts_code, start_date=start, end_date=end)
    start = ValidStartdate(start, df)
    dates = pd.bdate_range(start=start, end=end, freq='BM').strftime("%Y%m%d").tolist()
    num = len(dates)
    totalcount = 0
    if printable:
        print(num)
    dates[0] = ValidMonthDate(dates[0], df)
    initvalue = float(df.loc[df['trade_date'].isin([dates[0]]), 'close'])
    for i in range(num):
        target = fixed_investment*(i+1) # 每周定投固定数额累计
        dates[i] = ValidMonthDate(dates[i], df) 
        # print(dates[i])
        value = float(df.loc[df['trade_date'].isin([dates[i]]), 'close']) # 当前收盘值
        count = int(fixed_investment/value+0.5) # 当期应购数量
        totalcount = totalcount + count # 当前已拥有数量
        equity = totalcount*value # 当前净值
        income = equity - target - target*feesrate # 收益绝对值
        incomepercent = income / target * 100 # 定投收益百分比
        initpercent = (value-initvalue)/initvalue *100 # 相对初始收益百分比
        if printable: 
            print("%2d %s: %5d\t%.3f\t%d\t%.2f  \t%.2f    \t%.2f%%\t\t%.2f%%"%(i, dates[i], target, value, count, equity, income, incomepercent, initpercent))
    return incomepercent, start
        
MonthlyInvestments(ts_code, start, end, printable = True)

In [ ]:
def allweeklyincomepercent():
    print("start: ", start)
    print("end:   ", end)
    for k, v in alldict.items():
        # print("%s\t: \t %s"%(k,v))
        income, startdate = WeeklyInvestments(v, start, end)
        print('%s\t: \t%.2f%%\t\tfrom %s'%(k, income, startdate))
              
allweeklyincomepercent()

In [ ]:
def allmonthlyincomepercent():
    print("start: ", start)
    print("end:   ", end)
    for k, v in alldict.items():
        # print("%s\t: \t %s"%(k,v))
        income, startdate = MonthlyInvestments(v, start, end)
        print('%s\t: \t%.2f%%\t\tfrom %s'%(k, income, startdate))
              
allmonthlyincomepercent()